In [19]:
from langchain.llms import Ollama
# model_used = "starling-lm"
model_used = "llama2"
llm = Ollama(model=model_used)

In [20]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [21]:
chain = prompt | llm 

In [22]:
from langchain_core.output_parsers import StrOutputParser


output_parser = StrOutputParser()

In [23]:
chain = prompt | llm | output_parser

In [24]:
chain.invoke({"input": "how can langsmith help with testing?"})

'\nAs a world-class technical documentation writer, I can assist with testing in several ways:\n\n1. Content Creation: Langsmith can help create detailed and comprehensive test plans, test cases, and test scripts for various software applications, systems, and features. This includes creating user interface (UI) test cases, functional test cases, performance test cases, and security test cases.\n2. Testing Documentation: I can assist in creating documentation that outlines the testing process, including the test environment, test data, test scenarios, and test results. This documentation can help ensure consistency and repeatability of tests, and provide a clear understanding of the testing process for developers, testers, and other stakeholders.\n3. Test Automation: Langsmith can help automate repetitive and mundane testing tasks using tools like Selenium, Appium, or Pytest. By automating these tasks, we can free up time for more critical testing activities and reduce the risk of huma

In [25]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [26]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [27]:
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [29]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, langsmith can help with testing by allowing users to visualize their test results.'

In [30]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [31]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


LangSmith can help with testing in several ways:

1. Visualization: LangSmith provides a visual interface for reviewing and annotating runs, which can help identify areas of the chain that need improvement.
2. Automatic evaluation metrics: LangSmith includes evaluators that can be specified when initiating a test run, providing automatic metrics for evaluating the results.
3. Human evaluation: LangSmith makes it easy to manually review and annotate runs through annotation queues, allowing for subjective assessments of creativity or humor, as well as validation of auto-evaluated runs.
4. Monitoring: LangSmith can be used to monitor applications in much the same way that it was used for debugging, logging traced runs and visualizing latency and token usage statistics.
5. Exporting datasets: LangSmith makes it easy to curate datasets, which can be exported for use in other contexts, such as OpenAI Evals or fine-tuning with FireworksAI.

Overall, LangSmith simplifies dataset uploading, te